# Prep variables for HydroTas 2020-2021 workplan:
- Skill assessment
  - rainfall, surface temp and surface wind over Australia region and Tasmania region
  - Assess skill as function of start month and ensemble size
- UNSEEN
  - Tasmanian rainfall and Melbourne surface temperature

In [ ]:
import glob
import cftime
import geopandas
import numpy as np
import xarray as xr
from dask.distributed import Client
from dask_jobqueue import PBSCluster

In [ ]:
walltime = '02:00:00'
cores = 12
memory = '24GB'

cluster = PBSCluster(processes=1,
                     walltime=str(walltime), cores=cores, memory=str(memory),
                     job_extra=['-l ncpus='+str(cores),
                                '-l mem='+str(memory),
                                '-P ux06',
                                '-l jobfs=100GB',
                                '-l storage=gdata/xv83+gdata/v14+scratch/v14'],
                     local_directory='$PBS_JOBFS',
                     header_skip=["select"])

In [ ]:
cluster.scale(jobs=4)
client = Client(cluster)
client

# Create Australia land mask on CAFE grid

In [ ]:
NRM = geopandas.read_file('data/NRM_clusters/NRM_clusters.shp')
NRM_regions = {number: name for number, name in zip(list(NRM.OBJECTID), list(NRM.label))}
NRM_mask = regionmask.Regions_cls(name='NRM_regions', 
                                  numbers=list(NRM.OBJECTID), 
                                  names=list(NRM.label), 
                                  abbrevs=list(NRM.code), 
                                  outlines=list(NRM.geometry))
NRM_mask = NRM_mask.mask(da, lon_name=doppyo.utils.get_lon_name(da), 
                         lat_name=doppyo.utils.get_lat_name(da))

NRM_sub = geopandas.read_file("NRM_sub_clusters/NRM_sub_clusters.shp")
NRM_sub_regions = {number: name for number, name in zip(list(NRM_sub.OBJECTID), list(NRM_sub.label))}
NRM_sub_mask = regionmask.Regions_cls(name='NRM_sub_regions', 
                                      numbers=list(NRM_sub.index), 
                                      names=list(NRM_sub.label), 
                                      abbrevs=list(NRM_sub.code), 
                                      outlines=list(NRM_sub.geometry))
NRM_sub_mask = NRM_sub_mask.mask(da, lon_name=doppyo.utils.get_lon_name(da), 
                                 lat_name=doppyo.utils.get_lat_name(da))

region_masks = {'South-East Australia NRM region': xr.where((NRM_mask == 4) | (NRM_mask == 7), 1, np.nan),
                'Australia NRM region':            xr.where(NRM_mask.notnull(), 1, np.nan),

# Global parameters

In [ ]:
'Melbourne grid box': point_mask(da, (144.96, -37.81))
'Tasmania grid box':  point_mask(da, (146.5,  -42))
'Australia NRM region':            xr.where(NRM_mask.notnull(), 1, np.nan)

In [ ]:
REGIONS = {'AUS': [-44, -10, 112, 155],
           'TAS': [],
           'MEL': []}